In [ ]:
!pip install torch torchvision
!pip install open3d
!pip install gradio
# or
!pip install streamlit

In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())

# L3DG: Latent 3D Gaussian Diffusion -  Implementation and Demo
---
### Theoretical Background:

This code implements a simplified version of the L3DG approach for generative 3D modeling. It involves two key stages:

#### 1. Latent Compression via VQ-VAE:
- A VQ-VAE is used to compress complex 3D point cloud data
into a lower-dimensional latent space.
- Vector quantization discretizes the latent representation,
reducing complexity while preserving structural information.

#### 2. Latent Diffusion Modeling:
- A diffusion model operates in the learned latent space.
- It gradually denoises latent vectors, enabling generation
of new, realistic 3D structures from random noise.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

# -----------------------------
# Synthetic Data Generation
# -----------------------------
class Synthetic3DDataset(Dataset):
    def __init__(self, num_samples=1000, num_points=1024):
        # Each sample is a point cloud of shape (num_points, 3)
        self.data = [np.random.rand(num_points, 3).astype(np.float32) for _ in range(num_samples)]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Normalize each point cloud to [0, 1]
        return torch.tensor(self.data[idx])

# -----------------------------
# VQ-VAE Components
# -----------------------------
# Encoder: maps 3D point cloud to a latent vector
class Encoder(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=128, latent_dim=64):
        super(Encoder, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim)
        )

    def forward(self, x):
        # x: [B, N, 3]
        B, N, _ = x.shape
        x = x.view(B * N, -1)
        z = self.net(x)
        z = z.view(B, N, -1)  # reshape back to (B, N, latent_dim)
        # For simplicity, we aggregate over points via mean
        z = z.mean(dim=1)  # [B, latent_dim]
        return z

# Vector Quantizer: converts continuous latent vectors into discrete codebook entries
class VectorQuantizer(nn.Module):
    def __init__(self, num_embeddings=512, embedding_dim=64, commitment_cost=0.25):
        super(VectorQuantizer, self).__init__()
        self.num_embeddings = num_embeddings
        self.embedding_dim = embedding_dim
        self.commitment_cost = commitment_cost

        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.embedding.weight.data.uniform_(-1/self.num_embeddings, 1/self.num_embeddings)

    def forward(self, z):
        # z: [B, latent_dim]
        # Compute distances between z and embedding vectors
        # Expand z: [B, 1, latent_dim] and embeddings: [1, num_embeddings, latent_dim]
        z_expanded = z.unsqueeze(1)
        embeddings = self.embedding.weight.unsqueeze(0)
        distances = torch.sum((z_expanded - embeddings)**2, dim=2)  # [B, num_embeddings]

        # Get the nearest embedding indices
        encoding_indices = torch.argmin(distances, dim=1)
        z_q = self.embedding(encoding_indices)

        # Compute the loss for the VQ-VAE
        loss = torch.mean((z_q.detach() - z)**2) + self.commitment_cost * torch.mean((z_q - z.detach())**2)
        # Straight-through estimator
        z_q = z + (z_q - z).detach()
        return z_q, loss

# Decoder: reconstructs the point cloud from the quantized latent code
class Decoder(nn.Module):
    def __init__(self, latent_dim=64, hidden_dim=128, output_dim=3, num_points=1024):
        super(Decoder, self).__init__()
        self.num_points = num_points
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_points * output_dim)
        )

    def forward(self, z):
        # z: [B, latent_dim]
        x_recon = self.net(z)
        x_recon = x_recon.view(z.size(0), self.num_points, -1)  # [B, num_points, 3]
        return x_recon

# Full VQ-VAE Model
class VQVAE(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=128, latent_dim=64, num_embeddings=512, num_points=1024):
        super(VQVAE, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, latent_dim)
        self.vq = VectorQuantizer(num_embeddings, latent_dim)
        self.decoder = Decoder(latent_dim, hidden_dim, input_dim, num_points)

    def forward(self, x):
        z = self.encoder(x)
        z_q, vq_loss = self.vq(z)
        x_recon = self.decoder(z_q)
        return x_recon, vq_loss

# -----------------------------
# Training the VQ-VAE
# -----------------------------
def train_vqvae(model, dataloader, epochs=10, lr=1e-3, device='cpu'):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.to(device)

    for epoch in range(epochs):
        total_loss = 0
        for batch in dataloader:
            batch = batch.to(device)  # [B, num_points, 3]
            optimizer.zero_grad()
            recon, vq_loss = model(batch)
            # Reconstruction loss: L2 loss
            recon_loss = ((recon - batch) ** 2).mean()
            loss = recon_loss + vq_loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs} Loss: {total_loss/len(dataloader):.4f}")

# -----------------------------
# Running the VQ-VAE Training
# -----------------------------
if __name__ == '__main__':
    # Prepare data
    dataset = Synthetic3DDataset(num_samples=1000, num_points=1024)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    # Instantiate model
    vqvae_model = VQVAE(input_dim=3, hidden_dim=128, latent_dim=64, num_embeddings=512, num_points=1024)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Train VQ-VAE
    train_vqvae(vqvae_model, dataloader, epochs=20, lr=1e-3, device=device)

    # Save the trained VQ-VAE for later use
    torch.save(vqvae_model.state_dict(), "vqvae_model.pth")

In [ ]:
# -----------------------------
# Diffusion Model Components
# -----------------------------
class LatentDiffusion(nn.Module):
    def __init__(self, latent_dim=64, hidden_dim=128):
        super(LatentDiffusion, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim)
        )

    def forward(self, z_noisy):
        # Predict the "denoised" latent code
        return self.net(z_noisy)

# -----------------------------
# Diffusion Training Utilities
# -----------------------------
def add_noise(z, noise_level):
    noise = torch.randn_like(z) * noise_level
    return z + noise

def train_diffusion(model, vqvae, dataloader, epochs=10, lr=1e-3, device='cpu'):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    vqvae.to(device)
    vqvae.eval()  # Freeze the VQ-VAE during diffusion training

    for epoch in range(epochs):
        total_loss = 0
        for batch in dataloader:
            batch = batch.to(device)
            # Encode the clean latent codes with VQ-VAE encoder (we do not quantize here)
            with torch.no_grad():
                z_clean = vqvae.encoder(batch)
            # Add noise to the latent codes
            noise_level = 0.1  # You can adjust this schedule
            z_noisy = add_noise(z_clean, noise_level)
            # Model predicts the clean latent code from the noisy version
            z_pred = model(z_noisy)
            loss = ((z_pred - z_clean)**2).mean()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Diffusion Epoch {epoch+1}/{epochs} Loss: {total_loss/len(dataloader):.4f}")

# -----------------------------
# Running the Diffusion Model Training
# -----------------------------
if __name__ == '__main__':
    # Reload VQ-VAE model (if not already in memory)
    vqvae_model = VQVAE(input_dim=3, hidden_dim=128, latent_dim=64, num_embeddings=512, num_points=1024)
    vqvae_model.load_state_dict(torch.load("vqvae_model.pth", map_location=device))
    vqvae_model.to(device)

    # Instantiate and train the diffusion model
    diffusion_model = LatentDiffusion(latent_dim=64, hidden_dim=128)
    train_diffusion(diffusion_model, vqvae_model, dataloader, epochs=20, lr=1e-3, device=device)

    # Save the trained diffusion model for later sampling
    torch.save(diffusion_model.state_dict(), "diffusion_model.pth")

In [ ]:
import torch
import open3d as o3d
import plotly.graph_objects as go

def sample_latent(diffusion_model, latent_dim=64, noise_level=0.1, device='cpu'):
    print("\n Sampling latent code using diffusion model...")
    # Start with random noise
    z_noisy = torch.randn(1, latent_dim).to(device) * noise_level
    print("\n Random noise vector shape:", z_noisy.shape)
    # Use the diffusion model to predict a clean latent code
    with torch.no_grad():
        z_sampled = diffusion_model(z_noisy)
    print("\n Sampled latent vector shape:", z_sampled.shape)
    return z_sampled

def render_point_cloud(points_np):
    print("\n Rendering point cloud...")
    # Extract x, y, z coordinates from the numpy array
    x, y, z = points_np[:, 0], points_np[:, 1], points_np[:, 2]

    # Compute the distance from the origin for each point to use for color mapping
    distances = np.sqrt(x**2 + y**2 + z**2)

    # Create the Plotly figure with improved styling
    fig = go.Figure(data=[go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=3,            # Smaller marker size for denser clouds
            color=distances,   # Color by distance from the origin
            colorscale='Viridis',  # Use the Viridis colormap
            opacity=0.8,
            colorbar=dict(title="Distance")
        )
    )])

    # Update layout with titles and custom background
    fig.update_layout(
        title="3D Point Cloud Visualization",
        scene=dict(
            xaxis_title='X Axis',
            yaxis_title='Y Axis',
            zaxis_title='Z Axis',
            bgcolor='rgb(240,240,240)'
        ),
        margin=dict(l=0, r=0, b=0, t=30)
    )

    # Display the interactive Plotly figure
    fig.show()


# -----------------------------
# Sampling and Rendering Demo
# -----------------------------
# Set device based on availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Instantiate and load your models (ensure VQVAE and LatentDiffusion are defined)
print("\n Loading VQ-VAE model...")
vqvae_model = VQVAE(input_dim=3, hidden_dim=128, latent_dim=64, num_embeddings=512, num_points=1024)
vqvae_model.load_state_dict(torch.load("vqvae_model.pth", map_location=device))
vqvae_model.to(device)
print("\n VQ-VAE model loaded.")

print("\n Loading Latent Diffusion model...")
diffusion_model = LatentDiffusion(latent_dim=64, hidden_dim=128)
diffusion_model.load_state_dict(torch.load("diffusion_model.pth", map_location=device))
diffusion_model.to(device)
print("Latent Diffusion model loaded.\n")

# Sample a latent code using the diffusion model
z_sample = sample_latent(diffusion_model, latent_dim=64, noise_level=0.1, device=device)

# Decode the latent code to obtain a 3D point cloud
print("\n Decoding latent code into 3D point cloud...")
with torch.no_grad():
    point_cloud = vqvae_model.decoder(z_sample)
print("Decoded point cloud tensor shape:", point_cloud.shape)

# Convert the decoded point cloud tensor to numpy
point_cloud_np = point_cloud.squeeze(0).cpu().numpy()  # shape: [num_points, 3]
print("\n Final point cloud numpy array shape:", point_cloud_np.shape)

# Render the point cloud using the Plotly visualizer (works in Colab)
render_point_cloud(point_cloud_np)

print("\n Process complete.")
